In [3]:
import re
from json import dump
import requests
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [25]:
url = "https://www.australia-shoppings.com/malls-centres/victoria"
shopping_metadata = defaultdict(dict)
bs_object = BeautifulSoup(requests.get(url).text, "html.parser")

In [26]:
shopping_list = bs_object.find("ul" , {"class": "malls-list"})
pattern = r"(-?\d+(\.\d+)?),\s*(-?\d+(\.\d+)?)"
for li in shopping_list.find_all("li"):
    name = li.find("h3",{"class": "tit"}).text.strip()
    addr = li.find("p", {"class":"st"})
    suburb = addr.text.split(',')[1]
    match = re.findall(pattern, addr.text)
    coordinate = [float(match[0][0]),float(match[0][2])]
    shopping_metadata[name] = coordinate[0], coordinate[1], suburb

In [27]:
with open('../../data/raw/shopping_centre.json', 'w') as f:
    dump(shopping_metadata, f)

In [28]:
shopping = pd.read_json('../../data/raw/shopping_centre.json').transpose()

In [29]:
shopping = shopping.reset_index()
shopping.columns = ['name', 'latitude', 'longitude', 'suburb']

In [39]:
shopping.to_csv('../../data/curated/shopping_centre.csv')